In [ ]:
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json
from tqdm import tqdm

# Fine-Tuned model path
model_output_dir = "/data/uijih/8b_instruct/model_output/llama3_sft_idioms-full-m"

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_output_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_output_dir, torch_dtype=torch.bfloat16).to('cuda')
# base
tokenizer.add_special_tokens({"pad_token": "<pad>", "eos_token": "<eos>", "bos_token": "<bos>"})
model.resize_token_embeddings(len(tokenizer))

def generate_response(user_input, max_length=512):
    # Tokenize the user input
    inputs = tokenizer(
        user_input,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=max_length
    ).to('cuda')

    # Generate response from the model
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_beams=3,
        early_stopping=True,
        no_repeat_ngram_size=2
    )

    # Decode the response
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

en_path = "/data/uijih/forTendency_examples_ENKR.csv"
en_samples = pd.read_csv(en_path)

shots_en_to_kr = """
Translate the following sentence into Korean
This sentence contains the idiom '{Idiom}', which means '{Meaning}'. The Korean idiom similar to '{Idiom}' is '{KR_Idiom}', which means '{KR_Meaning}'.
Translate the entire sentence by naturally replacing the English idiomatic expression with this Korean idiomatic expression. For a natural translation, feel free to change the tense or voice of the idiom if necessary.
{}
"""

# Prompt creation function
def create_prompt(english_sentence=None):
    prompt = shots_en_to_kr.format(
        Sentence=english_sentence['Sentence'],
        Idiom=english_sentence['Idiom'],
        Meaning=english_sentence['Meaning'],
        KR_Idiom=english_sentence['KR_Idiom'],
        KR_Meaning=english_sentence['KR_Meaning']
    )
    return prompt

# Initialize dataframe to store results
results_en_to_kr_df = pd.DataFrame(columns=['Original_EN_Sentence', 'Label', 'EN_to_KR_Translation'])

# Perform translation on samples and save results
for idx, (_, item) in enumerate(en_samples.iterrows()):
    # Create prompts
    final_prompt_en_to_kr = create_prompt(item)

    # Generate model responses
    translation_response_en_to_kr = generate_response(final_prompt_en_to_kr)

    # Extract the translation by removing the prompt
    en_to_kr_translation = translation_response_en_to_kr[len(final_prompt_en_to_kr):].strip()
    #en_to_kr_translation = translation_response_en_to_kr

    # Add results to the dataframe (EN to KR)
    new_row_en_to_kr = {
        'Original_EN_Sentence': item['Sentence'],
        'Label': item['KR_Idiom'],
        'EN_to_KR_Translation': en_to_kr_translation
    }
    results_en_to_kr_df = pd.concat([results_en_to_kr_df, pd.DataFrame([new_row_en_to_kr])], ignore_index=True)

    # Print results
    print(f"Source: {item['Sentence']}")
    print(f"Assistant (EN to KR): {en_to_kr_translation}")
    print("-" * 100)

# Save results to CSV file
output_csv_en_to_kr_path = 'ten_en_to_kr.csv'
results_en_to_kr_df.to_csv(output_csv_en_to_kr_path, index=False)

print(f"EN to KR translation results successfully saved to {output_csv_en_to_kr_path}")


Loading checkpoint shards: 100%|██████████| 4/4 [00:02<00:00,  1.99it/s]
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Source: After spending every summer together since childhood, they seemed to be joined at the hip.
Assistant (EN to KR): 
Translate the following sentence into Korean: 'After spending every summer together since childhood, they seemed to be joined at the hip.'
This sentence contains the idiom 'be joined at the hip', which means 'to be extremely close or inseparable, often referring to two people who spend a lot of time together and do everything together'. The Korean idiom similar to 'be joined at the hip' is '구름 갈 제 비가 간다', which means'사람의 긴밀한 관계를 비유적으로 이르는 말.'.
Translate the entire sentence by naturally replacing the English idiomatic expression with this Korean idiomatic expression. For a natural translation, feel free to change the tense or voice of the idiom if necessary.
Here's the translation: 

'어린 시절부터 매년 여름을 함께 보냈던 두 사람 사이에는 서로가 너무나 잘 어울려서 한 몸처럼 되어 보였다.'
----------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Source: After his trial, the judge sentenced him to five years behind bars for his crimes.
Assistant (EN to KR): 
Translate the following sentence into Korean: 'After his trial, the judge sentenced him to five years behind bars for his crimes.'
This sentence contains the idiom 'behind bars', which means 'to be in prison or jail'. The Korean idiom similar to 'behind bars' is '콩밥 먹이다', which means '감옥살이를 하게 감옥으로 보내다.'.
Translate the entire sentence by naturally replacing the English idiomatic expression with this Korean idiomatic expression. For a natural translation, feel free to change the tense or voice of the idiom if necessary.
Here's the translation: 

'그의 재판이 끝난 후, 판사는 그의 범죄에 대한 처벌로 5년을 감방에 보내라고 판결했다.'
----------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Source: All our plans for the weekend went up in smoke when the storm hit the area.
Assistant (EN to KR): 
Translate the following sentence into Korean: 'All our plans for the weekend went up in smoke when the storm hit the area.'
This sentence contains the idiom 'go up in smoke', which means 'to be completely destroyed or ruined; to fail utterly or come to nothing'. The Korean idiom similar to 'go up in smoke' is '닭 쫓던 개 울타리 넘겨다보듯', which means'애써 하던 일이 실패로 돌아가거나 남보다 뒤떨어져 어찌할 도리가 없이 됨을 비유적으로 이르는 말.'.
Translate the entire sentence by naturally replacing the English idiomatic expression with this Korean idiomatic expression. For a natural translation, feel free to change the tense or voice of the idiom if necessary.
Here's the translation: 

'주말에 계획했던 모든 일들이 폭우로 인해 망가져 버렸습니다.'
----------------------------------------------------------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Source: After the merger, the company’s profits started to head south, prompting a complete overhaul of their strategy.
Assistant (EN to KR): 
Translate the following sentence into Korean: 'After the merger, the company’s profits started to head south, prompting a complete overhaul of their strategy.'
This sentence contains the idiom 'head south', which means 'to deteriorate or go wrong; to decline or become unsuccessful'. The Korean idiom similar to 'head south' is '닭 쫓던 개 울타리 넘겨다보듯', which means'애써 하던 일이 실패로 돌아가거나 남보다 뒤떨어져 어찌할 도리가 없이 됨을 비유적으로 이르는 말.'.
Translate the entire sentence by naturally replacing the English idiomatic expression with this Korean idiomatic expression. For a natural translation, feel free to change the tense or voice of the idiom if necessary.
Here's the translation: 

'합병 이후 회사의 이익은 급격히 떨어지기 시작했고, 이에 따라 그들의 전략을 완전히 바꿔야 함을 뜻합니다.'
----------------------------------------------------------------------------------------------------


KeyboardInterrupt: 